In [1]:
# Download latest FFmpeg static build.
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path

!which ffmpeg

/usr/bin/ffmpeg


In [2]:
# imports for parsing ffprobe
import subprocess as sp
import shlex
import json

In [3]:
# specify path for films folder
# this determines where the program looks for films and issues with them
films_folder = "Exercise3_Films"

# create a list of paths for the films
import glob
films = glob.glob(films_folder+"/*")

# create list of film names
from os import walk
filmnames = next(walk(films_folder), (None, None, []))[2]  # [] if no file

In [4]:
# log problem (line) in filmissues.txt
def log_issue(text_to_append):
    """Append given text as a new line at the end of file"""
    # Open the file in append & read mode ('a+')
    # filmissues.txt placed in root dir
    with open('filmissues.txt', "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)

In [5]:
# fills filmissues.txt for one film
def check_specs(film, filmname):
    # Execute ffprobe (to show streams), and get the output in JSON format
    data = sp.run(shlex.split(f'ffprobe -loglevel error -show_streams -of json {film}'), capture_output=True).stdout

    # Convert data from JSON string to dictionary
    d = json.loads(data)
    
    log_issue("Problems with " + filmname)
    log_issue("")
    
    if '.mp4' not in film:
        log_issue("x Video Format (Container): " + film.split(".",1)[1] )
    # video codec
    if d["streams"][0]["codec_name"] != 'h264':
        log_issue("x Video Codec: " + d["streams"][0]["codec_name"])
    # frame rate
    if d["streams"][0]["avg_frame_rate"] != '25000/1001' and d["streams"][0]["avg_frame_rate"] != '25/1':
        log_issue("x Frame Rate: " + d["streams"][0]["avg_frame_rate"])
    # aspect ratio
    if d["streams"][0]["display_aspect_ratio"] != '16:9':
        log_issue("x Aspect Ratio: " + d["streams"][0]["display_aspect_ratio"])
    # resolution
    if d["streams"][0]["width"] != 640 or d["streams"][0]["height"] != 360:
        log_issue("x Resolution: " + str(d["streams"][0]["width"]) + "x" + str(d["streams"][0]["height"]))
    # bit rate
    if int(d["streams"][0]["bit_rate"]) < 2000000 or  int(d["streams"][0]["bit_rate"]) > 5000000:
        log_issue("x Video Bit Rate: " + d["streams"][0]["bit_rate"])
    # audio codec
    if d["streams"][1]["codec_name"] != 'aac':
        log_issue("x Audio Codec: " + d["streams"][1]["codec_name"])
    # audio bit rate
    if int(d["streams"][1]["bit_rate"]) > 256000:
        log_issue("x Audio Bit Rate: " + d["streams"][1]["bit_rate"])
    # audio channels
    try: 
        if d["streams"][1]["channel_layout"] != 'stereo':
            log_issue("x Channel Layout: " + d["streams"][1]["channel_layout"])
    except:
        if d["streams"][1]["channels"] != 2:
            log_issue("x Channels: " + d["streams"][1]["channels"])
    
    log_issue("")
    log_issue("-----------------------------------")
    log_issue("")

In [6]:
# clears anything in filmissues.txt
open("filmissues.txt", "w").close()

# loops over films in dir, uses check specs to populate filmissues.txt
# filmissues.txt placed in root dir
for i in range(len(films)):
    check_specs(films[i], filmnames[i])

In [7]:
# the full ffmpeg function

# !ffmpeg -i Exercise3_Films/Cosmos_War_of_the_Planets.mp4 -aspect:v "16:9" -s '640x360' -c:v h264 -c:a aac -b:a 256K -b:v 3.5M -minrate:v 2M -maxrate:v 5M -ac 2 -r 25 output.mp4
                            

In [8]:
# fix movies
def fix_format(film):
    # Execute ffprobe (to show streams), and get the output in JSON format
    data = sp.run(shlex.split(f'ffprobe -loglevel error -show_streams -of json {film}'), capture_output=True).stdout

    # Convert data from JSON string to dictionary
    d = json.loads(data)
    
    # ffcode builder list
    ffcode = [f'ffmpeg -i {film}']
    
    # aspect ratio
    if d["streams"][0]["display_aspect_ratio"] != '16:9':
        ffcode.append('-aspect:v "16:9"')
    # resolution
    if d["streams"][0]["width"] != 640 or d["streams"][0]["height"] != 360:
        ffcode.append('-s "640x360"')
        
    # video codec
    if d["streams"][0]["codec_name"] != 'h264':
        ffcode.append('-c:v h264')
        
    # audio codec
    if d["streams"][1]["codec_name"] != 'aac':
        ffcode.append('-c:a aac')
        
    # audio bit rate
    if int(d["streams"][1]["bit_rate"]) > 256000:
        ffcode.append('-b:a 256k')
    
    # video bit rate
    if int(d["streams"][0]["bit_rate"]) < 2000000 or  int(d["streams"][0]["bit_rate"]) > 5000000:
        ffcode.append('-b:v 3.5m -minrate:v 2m -maxrate:v 5m')
        
    # audio channels
    try:
        if d["streams"][1]["channel_layout"] != 'stereo':
            ffcode.append('-ac 2')
    except:
        if d["streams"][1]["channels"] != 2:
            ffcode.append('-ac 2')
        
    # frame rate
    if d["streams"][0]["avg_frame_rate"] != '25000/1001' and d["streams"][0]["avg_frame_rate"] != '25/1':
        ffcode.append('-r 25')
    
    # ensure container is mp4 and _formatOK is there
    ffcode.append(film.split('.')[0] + '_formatOK' + '.mp4')
    
    # joing ffcode list
    ffcode = ' '.join(ffcode)
    
    # subprocess call our ffmpeg line in shell
    sp.call(ffcode, shell=True)

In [9]:
# loops over films in dir, uses fix format to correct films
# new films (_formatOK) placed in film folder
for i in range(len(films)):
    fix_format(films[i])
    print(films[i] + " done.")

Exercise3_Films/Cosmos_War_of_the_Planets.mp4 done.
Exercise3_Films/Last_man_on_earth_1964.mov done.
Exercise3_Films/The_Hill_Gang_Rides_Again.mp4 done.
Exercise3_Films/Voyage_to_the_Planet_of_Prehistoric_Women.mp4 done.
Exercise3_Films/The_Gun_and_the_Pulpit.avi done.
